In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 01_register_realtime_model
# MAGIC 
# MAGIC - 베이스 모델 확인
# MAGIC - RealtimeRealScoreModel 등록
# MAGIC - MLflow Experiment 설정
# MAGIC - GitHub Actions 환경변수 지원
# MAGIC - Staging stage 자동 전환

# COMMAND ----------

import os
import sys
import logging
from datetime import datetime
sys.path.append("/Workspace/Users/1dt003@msacademy.msai.kr/team4-CICD/src")

from azure.storage.blob import BlobServiceClient
import mlflow
from mlflow.tracking import MlflowClient
from src.realtime_model import RealtimeRealScoreModel

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# ----------------------
# 환경변수 및 기본값 설정
# ----------------------
EXPERIMENT_NAME = os.getenv("REALTIME_EXPERIMENT", "/Workspace/Users/1dt003@msacademy.msai.kr/team4_inference_experiment")
BASE_MODEL_NAME = os.getenv("BASE_MODEL_NAME", "team4-pred-model")
REALTIME_MODEL_NAME = os.getenv("REALTIME_MODEL_NAME", "team4-realtime-model")
ENVIRONMENT = os.getenv("ENVIRONMENT", "Staging")  # default stage

# MLflow 설정
mlflow.set_experiment(EXPERIMENT_NAME)
client = MlflowClient()

print(f"✅ 실험: {EXPERIMENT_NAME}")
print(f"✅ 베이스 모델: {BASE_MODEL_NAME}")
print(f"✅ 실시간 모델: {REALTIME_MODEL_NAME}")

# COMMAND ----------

# 1️⃣ 베이스 모델 확인
try:
    base_model_versions = client.search_model_versions(f"name='{BASE_MODEL_NAME}'")
    if base_model_versions:
        latest_version = max(base_model_versions, key=lambda x: int(x.version))
        base_model_uri = f"models:/{BASE_MODEL_NAME}/{latest_version.version}"
        print(f"✅ 베이스 모델 발견: {base_model_uri}")
        print(f"   - 버전: {latest_version.version}")
        print(f"   - 스테이지: {latest_version.current_stage}")
    else:
        print("❌ 베이스 모델을 찾을 수 없습니다.")
        base_model_uri = None
except Exception as e:
    print(f"❌ 베이스 모델 확인 중 오류: {e}")
    base_model_uri = None

# COMMAND ----------

# 2️⃣ Realtime 모델 등록
realtime_run_id = None
realtime_model_version = None

if base_model_uri:
    try:
        with mlflow.start_run(run_name=f"{REALTIME_MODEL_NAME}_deployment_{ENVIRONMENT}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
) as run:
            print("🚀 실시간 추론 모델 등록 시작...")

            mlflow.pyfunc.log_model(
                artifact_path="realtime_model",
                python_model=RealtimeRealScoreModel(),
                artifacts={"model": base_model_uri},
                registered_model_name=REALTIME_MODEL_NAME,
                pip_requirements=[
                    "mlflow>=2.0.0",
                    "pandas>=1.3.0",
                    "numpy>=1.21.0",
                    "scipy>=1.7.0",
                    "scikit-learn>=1.0.0",
                    "azure-storage-blob>=12.0.0"
                ]
            )

            realtime_run_id = run.info.run_id
            print(f"✅ 실시간 모델 등록 완료! Run ID: {realtime_run_id}")

        # -----------------------------
        # 3️⃣ Staging stage로 자동 전환
        # -----------------------------
        latest_versions = client.get_latest_versions(name=REALTIME_MODEL_NAME, stages=["None"])
        if latest_versions:
            # 최신 버전 선택
            realtime_model_version = max([int(v.version) for v in latest_versions])
            client.transition_model_version_stage(
                name=REALTIME_MODEL_NAME,
                version=realtime_model_version,
                stage="Staging",
                archive_existing_versions=False
            )
            print(f"🎉 실시간 모델 version {realtime_model_version}을 Staging으로 전환 완료")
        else:
            print("⚠️ 등록된 None stage 모델을 찾지 못했습니다. Staging 전환 실패")

    except Exception as e:
        print(f"❌ 모델 등록 실패: {e}")
        realtime_run_id = None
        realtime_model_version = None
else:
    print("❌ 베이스 모델이 없어서 실시간 모델을 생성할 수 없습니다.")

# COMMAND ----------

# 4️⃣ 최종 확인
if realtime_run_id:
    print(f"🎉 등록 완료! Run ID: {realtime_run_id}")
    if realtime_model_version:
        print(f"🎉 Staging version: {realtime_model_version}")
else:
    print("⚠️ 실시간 모델 등록 실패")


In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 02_stage_model
# MAGIC 
# MAGIC - MLflow에 등록된 Realtime 모델을 Staging으로 이동

# COMMAND ----------
import os


from mlflow.tracking import MlflowClient
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ----------------------
# 설정
# ----------------------
REALTIME_MODEL_NAME = os.getenv("REALTIME_MODEL_NAME","realtime-model-staging")
client = MlflowClient()

# COMMAND ----------

# 1️⃣ 최신 모델 버전 가져오기
try:
    latest_versions = client.get_latest_versions(REALTIME_MODEL_NAME, stages=["None"])
    
    if not latest_versions:
        print(f"❌ '{REALTIME_MODEL_NAME}'의 None 단계 모델을 찾을 수 없습니다.")
    else:
        latest_version = latest_versions[0]
        version_number = latest_version.version
        print(f"✅ 최신 모델 버전: {version_number}")
except Exception as e:
    print(f"❌ 모델 버전 확인 실패: {e}")

# COMMAND ----------

# 2️⃣ Staging으로 이동
try:
    client.transition_model_version_stage(
        name=REALTIME_MODEL_NAME,
        version=version_number,
        stage="Staging",
        archive_existing_versions=False
    )
    print(f"✅ 모델 버전 {version_number}을 Staging으로 이동 완료!")
except Exception as e:
    print(f"❌ 모델 스테이지 이동 실패: {e}")
